# 5. Select and Train a Model

In [ ]:
import joblib
housing_prepared = joblib.load("data/housing_prepared.pkl") # 노트북 나눠서 불러오기
housing_labels = joblib.load("data/housing_labels.pkl")
housing = joblib.load("data/housing_raw.pkl")

In [8]:
import pickle
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

# CombinedAttributesAdder 클래스 정의
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]  
        population_per_household = X[:, population_ix] / X[:, households_ix]  
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]  
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

# pickle로 저장된 CombinedAttributesAdder 객체 불러오기
with open('combined_attributes_adder.pkl', 'rb') as f:
    attr_adder = pickle.load(f)

# 이제 attr_adder를 사용하여 변환을 수행할 수 있다.


In [10]:
# 저장된 full_pipeline 불러오기
with open('full_pipeline.pkl', 'rb') as f:
    full_pipeline = pickle.load(f)

In [14]:
col_names = "total_rooms", "total_bedrooms", "population", "households" # 재지정
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names] # get the column indices

## Training and Evaluating on the Training Set

In [3]:
from sklearn.linear_model import LinearRegression  

# 선형 회귀 모델 객체 생성
lin_reg = LinearRegression()

# 변환된 데이터(housing_prepared)와 레이블(housing_labels)을 사용하여 모델 훈련
lin_reg.fit(housing_prepared, housing_labels)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [13]:
# 몇 가지 훈련 인스턴스에서 전체 전처리 파이프라인 시도

# 훈련 데이터에서 처음 5개의 인스턴스를 선택
some_data = housing.iloc[:5]  # housing 데이터에서 첫 5개 행을 선택
some_labels = housing_labels.iloc[:5]  # housing_labels에서 첫 5개 레이블을 선택

# 선택된 데이터에 대해 전체 전처리 파이프라인을 적용
some_data_prepared = full_pipeline.transform(some_data)  # 전체 전처리 파이프라인을 적용하여 데이터 준비

# 준비된 데이터로 선형 회귀 모델을 사용하여 예측 수행
print("Predictions:", lin_reg.predict(some_data_prepared))  # 선형 회귀 모델을 통해 예측값 출력

Predictions: [ 85657.90192014 305492.60737488 152056.46122456 186095.70946094
 244550.67966089]


In [16]:
print("Labels:", list(some_labels)) # 실제 값과 비교

Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


In [17]:
some_data_prepared

array([[-0.94135046,  1.34743822,  0.02756357,  0.58477745,  0.64037127,
         0.73260236,  0.55628602, -0.8936472 ,  0.01739526,  0.00622264,
        -0.12112176,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.17178212, -1.19243966, -1.72201763,  1.26146668,  0.78156132,
         0.53361152,  0.72131799,  1.292168  ,  0.56925554, -0.04081077,
        -0.81086696,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.26758118, -0.1259716 ,  1.22045984, -0.46977281, -0.54513828,
        -0.67467519, -0.52440722, -0.52543365, -0.01802432, -0.07537122,
        -0.33827252,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.22173797, -1.35147437, -0.37006852, -0.34865152, -0.03636724,
        -0.46761716, -0.03729672, -0.86592882, -0.59513997, -0.10680295,
         0.96120521,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.43743108, -0.63581817, -0

In [22]:
from sklearn.metrics import mean_squared_error  # mean_squared_error를 임포트

# 선형 회귀 모델을 사용하여 housing_prepared 데이터에 대한 예측 수행
housing_predictions = lin_reg.predict(housing_prepared)

# 실제값(housing_labels)과 예측값(housing_predictions)을 사용하여 평균 제곱 오차(MSE) 계산
lin_mse = mean_squared_error(housing_labels, housing_predictions)

# 평균 제곱 오차(MSE)의 제곱근을 구하여 루트 평균 제곱 오차(RMSE) 계산
lin_rmse = np.sqrt(lin_mse)

# 계산된 루트 평균 제곱 오차(RMSE) 출력
lin_rmse


np.float64(68627.87390018745)

In [23]:
from sklearn.metrics import mean_absolute_error  # mean_absolute_error를 임포트

# 모델의 예측값과 실제값을 사용하여 평균 절대 오차 계산
lin_mae = mean_absolute_error(housing_labels, housing_predictions)

# 계산된 평균 절대 오차 출력
lin_mae


49438.66860915801

In [20]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [24]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

np.float64(0.0)

## Better Evaluation Using Cross-Validation

In [26]:
from sklearn.model_selection import cross_val_score  # cross_val_score를 임포트

# 교차 검증을 사용하여 결정 트리 모델의 성능 평가
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)  # 10-겹 교차 검증, MSE를 음수로 반환

# 음수로 반환된 MSE 값을 양수로 변환하고, 제곱근을 구하여 RMSE 계산
tree_rmse_scores = np.sqrt(-scores)

# tree_rmse_scores는 각 폴드(fold)에서 계산된 RMSE를 담고 있다.


In [30]:
def display_scores(scores):
    # 주어진 scores를 출력 (각각의 성능 점수, 예: RMSE 값들)
    print("Scores:", scores)
    # 성능 점수의 평균 계산 후 출력
    print("Mean:", scores.mean())
    # 성능 점수의 표준 편차 계산 후 출력
    print("Standard deviation:", scores.std())

# tree_rmse_scores 배열에 대해 display_scores 함수 실행
display_scores(tree_rmse_scores)


Scores: [72831.45749112 69973.18438322 69528.56551415 72517.78229792
 69145.50006909 79094.74123727 68960.045444   73344.50225684
 69826.02473916 71077.09753998]
Mean: 71629.89009727491
Standard deviation: 2914.035468468928


In [32]:
# 선형 회귀 모델에 대해 교차 검증을 수행하여 평균 제곱 오차(MSE)를 계산
# scoring="neg_mean_squared_error"는 MSE를 음수로 반환하기 때문에 이를 변환해야 함
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)

# 음수로 반환된 MSE 값을 양수로 변환하고, 제곱근을 구하여 RMSE 계산
lin_rmse_scores = np.sqrt(-lin_scores)

# RMSE 점수의 평균과 표준 편차를 출력
display_scores(lin_rmse_scores)

Scores: [71762.76364394 64114.99166359 67771.17124356 68635.19072082
 66846.14089488 72528.03725385 73997.08050233 68802.33629334
 66443.28836884 70139.79923956]
Mean: 69104.07998247063
Standard deviation: 2880.3282098180666


In [33]:
# 기본값이 Scikit-Learn 0.22에서 100으로 변경될 예정이므로,
# 미래를 대비해 n_estimators=100으로 지정
from sklearn.ensemble import RandomForestRegressor 

# 랜덤 포레스트 회귀 모델 생성 (n_estimators=100으로 100개의 결정 트리를 사용, random_state=42로 재현 가능하게 설정)
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)

# 모델 훈련: 준비된 데이터(housing_prepared)와 실제 값(housing_labels)을 사용하여 모델을 훈련시킴
forest_reg.fit(housing_prepared, housing_labels)


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [35]:
# 훈련된 랜덤 포레스트 모델을 사용하여 housing_prepared 데이터에 대한 예측 수행
housing_predictions = forest_reg.predict(housing_prepared)

# 실제값(housing_labels)과 예측값(housing_predictions)을 사용하여 평균 제곱 오차(MSE) 계산
forest_mse = mean_squared_error(housing_labels, housing_predictions)

# 평균 제곱 오차(MSE)의 제곱근을 구하여 루트 평균 제곱 오차(RMSE) 계산
forest_rmse = np.sqrt(forest_mse)

# 계산된 루트 평균 제곱 오차(RMSE) 출력
forest_rmse

np.float64(18650.698705770003)

In [ ]:
# 교차 검증을 사용하여 랜덤 포레스트 모델의 성능 평가
# scoring="neg_mean_squared_error"로 평균 제곱 오차(MSE)를 음수로 반환받음
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)

# 음수로 반환된 MSE 값을 양수로 변환하고, 제곱근을 구하여 루트 평균 제곱 오차(RMSE) 계산
forest_rmse_scores = np.sqrt(-forest_scores)

# RMSE 점수의 평균과 표준 편차를 출력
display_scores(forest_rmse_scores)


In [ ]:
# 교차 검증을 사용하여 선형 회귀 모델의 성능 평가
# scoring="neg_mean_squared_error"로 평균 제곱 오차(MSE)를 음수로 반환받음
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

# 음수로 반환된 MSE 값을 양수로 변환하고, 제곱근을 구하여 루트 평균 제곱 오차(RMSE) 계산
# pd.Series(np.sqrt(-scores))를 사용하여 RMSE의 통계적 요약 정보를 출력
pd.Series(np.sqrt(-scores)).describe()  # RMSE의 평균, 표준 편차, 최소값, 최대값 등을 포함한 요약 통계 출력


In [ ]:
from sklearn.svm import SVR  
# 선형 커널을 사용하여 Support Vector Regression 모델 생성
svm_reg = SVR(kernel="linear")

# 모델 훈련: 준비된 데이터(housing_prepared)와 실제 값(housing_labels)을 사용하여 모델을 훈련
svm_reg.fit(housing_prepared, housing_labels)

# 훈련된 모델을 사용하여 housing_prepared 데이터에 대해 예측 수행
housing_predictions = svm_reg.predict(housing_prepared)

# 실제값(housing_labels)과 예측값(housing_predictions)을 사용하여 평균 제곱 오차(MSE) 계산
svm_mse = mean_squared_error(housing_labels, housing_predictions)

# 평균 제곱 오차(MSE)의 제곱근을 구하여 루트 평균 제곱 오차(RMSE) 계산
svm_rmse = np.sqrt(svm_mse)

# 계산된 루트 평균 제곱 오차(RMSE) 출력
svm_rmse
